# Lab 3, part I: sensitivity analysis

This lab focuses on sensitivity analysis. After solving a problem with three constraints and four variables, we will change the problem slightly and re-solve it to evaluate how the solution and the objective function change. Then we'll use the problem's dual variables and reduced costs to arrive at the same result.

Consider the product mix problem: A factory produces 4 types of perfume by mixing 5 ingredients. Denote as $P=\{1,2,3,4\}$ the set of perfumes, and as $I=\{1,2,3,4,5\}$ the set of ingredients. The retail price for each liter of perfume $p\in P$ is $c_p$, defined in the table below:

|Perfume | 1 | 2 | 3 | 4 |
|--------|---|---|---|---|
|Retail price  |300|255|260|390|

One liter of perfume $p$ requires a quantity $a_{ip}$ of the ingredient $i$, also expressed in liters, as specified below:

|Ingredient\Perfume|1|2|3|4|
|-|-|-|-|-|
|1|0.01|0.05|0.07|0.04|
|2|0.34|0.45|0.36|0.51|
|3|0.08|0.06|0.12|0.12|
|4|0.55|0.35|0.29|0.32|
|5|0.02|0.09|0.16|0.01|

There is a finite amount (in liters) of each ingredient expressed by $b_i$, for all $i\in I$ and defined in the following table:

|Ingredient|1|2|3|4|5|
|----------|-|-|-|-|-|
|Availability|30|400|90|450|70|

Here are the tasks for this exercise:

1. Determine the amount of perfume to produce for each type in order to maximize the total revenue;
2. What could allow us to increase the total profit without changing the retail price $c_p$ and the percentage $a_{ip}$?
3. Find what constraint(s) are the most restrictive by changing one of the input parameters $b_i$;
4. Are all perfumes produced? If any of them is not produced, can we reduce its retail price so that it is worth to produce it? By how much?

## Solution

Task 1: __Determine the amount of perfume to produce for each type in order to maximize the total revenue.__

We must create an optimization problem for this purpose. First, define sets and parameters:

* Set $P = \{1,2,3,4\}$;
* Set $I = \{1,2,3,4,5\}$;
* Retail price $c = (300,255,260,390)^\top$;
* Availability $b = (30,400,90,450,70)^\top$;
* Composition $A = \left(
\begin{array}{rrrrr}
0.01 & 0.05 & 0.07 & 0.04\\
0.34 & 0.45 & 0.36 & 0.51\\
0.08 & 0.06 & 0.12 & 0.12\\
0.55 & 0.35 & 0.29 & 0.32\\
0.02 & 0.09 & 0.16 & 0.01\\
\end{array}
\right)$

There is only one class of variables, $x_i$ for each $i\in P$, i.e. one nonnegative variable for each type of perfume. The problem is then as follows:

$$
\begin{array}{lrrrrrr}
\max          & 300 x_1 &+  255 x_2 &+  260 x_3  &+ 390 x_4\\
\textrm{s.t.} &0.01 x_1 &+ 0.05 x_2 &+ 0.07 x_3 &+ 0.04 x_4 &\le & 30\\
              &0.34 x_1 &+ 0.45 x_2 &+ 0.36 x_3 &+ 0.51 x_4 &\le &400\\
              &0.08 x_1 &+ 0.06 x_2 &+ 0.12 x_3 &+ 0.12 x_4 &\le & 90\\
              &0.55 x_1 &+ 0.35 x_2 &+ 0.29 x_3 &+ 0.32 x_4 &\le &450\\
              &0.02 x_1 &+ 0.09 x_2 &+ 0.16 x_3 &+ 0.01 x_4 &\le & 70\\
              &x_1, &x_2, &x_3, &x_4 &\ge& 0,
\end{array}
$$

or, more concisely,

$$
\begin{array}{ll}
\min & c^\top x\\
\textrm{s.t.} & Ax \le b\\
              & x \ge 0.
\end{array}
$$

We implement it below using Python-MIP.

Because we will solve this problem more than once, we create a function that takes $A$, $b$, and $c$ as arguments and returns the optimal solution found, its objective function value, and the model itself.

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

In [1]:
import mip

c = [300, 255, 260, 390]
b = [30, 400, 90, 450, 70]

A = \
[[0.01, 0.05, 0.07, 0.04],
 [0.34, 0.45, 0.36, 0.51],
 [0.08, 0.06, 0.12, 0.12],
 [0.55, 0.35, 0.29, 0.32],
 [0.02, 0.09, 0.16, 0.01]]

def solve_productmix(A, b, c):
    """
    Solve the problem max{cx: Ax <= b, x >= 0} with A, b, c passed as arguments
    """

    n = len(c)
    k = len(b)

    # create model (TODO)
    m = mip.Model()
    
    # add variables (TODO)
    x = [ m.add_var() for j in range(n) ]

    # One constraint per ingredient (TODO)
    for i in range(k):
        m.add_constr(mip.xsum([A[i][j] * x[j] for j in range(n)]) <= b[i])
    
    # Objective function is a weighted sum of all x (TODO)
    m.objective = mip.maximize(mip.xsum([c[j] * x[j] for j in range(n)]))
    # Solve
    m.optimize()

    # Return a tuple containing model, solution, and objective value
    return (m, [x[j].x for j in range(n)], m.objective_value)


# Use "_" because we don't care about the model in this call
_, solution, objective = solve_productmix(A,b,c)

print("Solution:", solution)
print("Objective: {0:10.2f}".format(objective))

Solution: [573.115003808073, 89.74358974359002, 0.0, 323.0515359228229]
Objective:  320809.22


Task 2: __What could allow us to increase the total profit without changing the retail price $c_p$ and the composition $a_{ip}$?__

If the retail price and the composition cannot change, we can only modify the availability. For example, by increasing all values of $b$ by 10%, we can easily increase the revenue by the same percentage:

In [2]:
b2 = [b[i]*1.1 for i in I]

_, solution2, objective2 = solve_productmix(A,b2,c)

print("Solution:", solution2)
print(f"Objective: {objective2}, increase: {((objective2 - objective) / objective) * 100: 6.2f}%")

Solution: [630.4265041888804, 98.71794871794896, 0.0, 355.3566895151053]
Objective: 352890.1370906322, increase:  10.00%


Task 3: __Find which constraint is the most restrictive by changing the input parameters $b_i$ one by one.__

We can loop over each ingredient and solve a problem where one of the components is increased by 0.1. This will tell us which constraint could be relaxed to increase the total revenue, and possibly which one yields the greatest increase.

In [3]:
for i in I:
    b3 = b[:]    # Use [:] to make a copy of b, as otherwise modifying b3 would change b as well
    b3[i] += .1  # Make a tiny change to the right-hand side
    _, sol, obj = solve_productmix(A,b3,c)

    print(f"Objective: {obj:10.2f}, Increase: {obj - objective:8.2f}, {(obj - objective)/objective * 100:5.2f}%")

Objective:  320809.22, Increase:     0.00,  0.00%
Objective:  320828.41, Increase:    19.19,  0.01%
Objective:  321020.96, Increase:   211.75,  0.07%
Objective:  320821.10, Increase:    11.88,  0.00%
Objective:  320809.22, Increase:     0.00,  0.00%


However, there is no need to solve as many LPs as the constraints. This information is returned back to us with the __dual variables__ of the original problem. These give the unitary change in objective function due to a change in the right-hand side in each constraint.

In [4]:
m, sol, obj = solve_productmix(A,b,c)
# Check the resulting list with the values you obtained before.
print("Dual values:", [con.pi for con in m.constrs])

Dual values: [-0.0, 191.92688499619217, 2117.479055597866, 118.81188118811873, -0.0]


Task 4: __Are all perfumes produced? If any of them is not produced, can we increase its retail price so that it is worth to produce it? By how much?__

In the original solution, perfume 3 is not produced. Its retail price $c_3$, which is also its objective function coefficient, is 260. Let's try running a loop where we increase $c_3$ by 5 until $x_3>0$ in the optimal solution.

In [5]:
c_new = c[:]  # As we said before, this is to create a new list as a copy of c
sol = solution[:]  # Copy sol as well
while sol[2] <= 1e-5:  # Sol[2] is x_3 (remember that Python has 0-based indexing)
    c_new[2] += 5
    _, sol, obj = solve_productmix(A,b,c_new)
    print(f"With c_3 = {c_new[2]}: produce {sol[2]} liters of 3")

print (f"We need to set the price c_3 to {c_new[2]}")

With c_3 = 265: produce 0.0 liters of 3
With c_3 = 270: produce 0.0 liters of 3
With c_3 = 275: produce 0.0 liters of 3
With c_3 = 280: produce 0.0 liters of 3
With c_3 = 285: produce 0.0 liters of 3
With c_3 = 290: produce 0.0 liters of 3
With c_3 = 295: produce 0.0 liters of 3
With c_3 = 300: produce 0.0 liters of 3
With c_3 = 305: produce 0.0 liters of 3
With c_3 = 310: produce 0.0 liters of 3
With c_3 = 315: produce 0.0 liters of 3
With c_3 = 320: produce 0.0 liters of 3
With c_3 = 325: produce 0.0 liters of 3
With c_3 = 330: produce 0.0 liters of 3
With c_3 = 335: produce 0.0 liters of 3
With c_3 = 340: produce 0.0 liters of 3
With c_3 = 345: produce 0.0 liters of 3
With c_3 = 350: produce 0.0 liters of 3
With c_3 = 355: produce 0.0 liters of 3
With c_3 = 360: produce 116.60624892111176 liters of 3
We need to set the price c_3 to 360


Again, we don't need to solve that many problems. All we need is the __reduced cost__ of $x_3$. In Python-MIP, the reduced cost of a variable `x` is obtained as `x.rc`. Given that the third variable of our model is `m.vars[2]`,we need to print:

In [6]:
print(m.vars[2].rc)

-97.64661081492756


Notice anything? If we add 97.6466 to the original retail price $c_3$, we get $260 + 97.6466 = 357.6466$, which is between 355 and 360. Let's make a fine-grained comparison between two very close values of $c_3$: one at $260 + 97.64 = 357.64$ and another one at $260 + 97.65 = 357.65$. The prediction is that the first will give a value of zero for $x_3$, unlike the second.

In [7]:
c_test = c[:]

c_test[2] = c[2] + 97.64
_, sol, obj = solve_productmix(A,b,c_test)
print(f"before threshold: x_3 = {sol[2]}")

c_test[2] = c[2] + 97.65
_, sol, obj = solve_productmix(A,b,c_test)
print(f"after threshold: x_3 = {sol[2]}")

before threshold: x_3 = 0.0
after threshold: x_3 = 116.60624892111176
